# Plot various heatmaps of the integrated data  
[**Heatmap with temperature profile**](#temp)  
[**Heatmap with correlation matrix**](#corr)  
[**Heatmap with deviation from mean**](#mean)  

In [ ]:
%matplotlib widget
import DanMAX as DM
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [ ]:
fname = DM.getLatestScan()
aname = DM.getAzintFname(fname)
with h5py.File(aname,'r') as f:
    try:
        x = f['q'][:]
        Q = True
    except KeyError:
        x = f['2th'][:]
        Q = False
    I = f['I'][:]
meta = DM.getMetaData(fname)
t = meta['time']
T = meta['temp']
I0 = meta['I0']

# normalize
I = (I.T/I0).T


## Plot heatmap with temperature profile <a id='temp'></a>

In [ ]:
# initialize figure
fig = plt.figure()
fig.tight_layout()

# initialize grid and subplot with different size-ratios
grid = plt.GridSpec(1,2,width_ratios=[5,1]) #rows,columns
ax0, ax1 = [fig.add_subplot(gr) for gr in grid]

# set shared y-axis
ax1.sharey(ax0)

# set ticks on right y-axis
ax0.tick_params('y',right=True)
ax1.tick_params('y',right=True)

# remove left ticks labels from second subplot
ax1.tick_params('y',labelleft=False)

# set title
ax0.set_title(DM.getScan_id(fname))

# set axis labels
if Q:
    ax0.set_xlabel('q (A-1)')
else:
    ax0.set_xlabel('2theta (deg)')
ax0.set_ylabel('Time (s)')
ax1.set_xlabel('Temperature (K)')

# plot heatmap and temperature profile
ax0.pcolormesh(x,t,I, shading='nearest')
ax1.plot(T,t,'k')

## Plot heatmap and correlation matrix <a id='corr'></a>

In [ ]:
# calculate correlation matrix
corr = np.corrcoef(I)

fig, (ax0,ax1) = plt.subplots(1,2)
fig.set_figwidth(10)
ax0.pcolormesh(x,t,I, shading='nearest')
ax1.pcolormesh(t,t,np.abs(corr),shading='nearest', cmap='viridis')#,vmax=1)

ax1.set_aspect(1)

ax0.set_title('Heatmap')
ax1.set_title('Correlation map')
if Q:
    ax0.set_xlabel('q (A-1)')
else:
    ax0.set_xlabel('2theta (deg)')
ax1.set_xlabel('Time (s)')

ax0.set_ylabel('Time (s)')
ax1.set_ylabel('Time (s)')

fig.tight_layout()

## Plot heatmap and deviation from mean <a id='mean'></a>

In [ ]:
I[I<=0]=np.nan
# calculate deviation from mean pattern
dev_mean = (I - np.nanmean(I,axis=0))/I*100

fig, (ax0,ax1) = plt.subplots(1,2)
fig.set_figwidth(10)
ax0.pcolormesh(x,t,I, shading='nearest')
pcm = ax1.pcolormesh(x,t,dev_mean,shading='nearest', cmap='RdBu_r',vmax=5,vmin=-5)

ax0.set_title('Heatmap')
ax1.set_title('Deviation from mean')
if Q:
    ax0.set_xlabel('q (A-1)')
    ax1.set_xlabel('q (A-1)')
else:
    ax0.set_xlabel('2theta (deg)')
    ax1.set_xlabel('2theta (deg)')

ax0.set_ylabel('Time (s)')
ax1.set_ylabel('Time (s)')

fig.colorbar(pcm,ax=ax1,label='Deviation (%)')

fig.tight_layout()
